In [35]:
import sys
print(sys.executable)

c:\Users\Donggu Lee\AppData\Local\Programs\Python\Python310\python.exe


In [36]:
import numpy as np

In [37]:
WORLD_SIZE = 5
A_POS = [0, 1]
A_PRIME_POS = [4, 1]
B_POS = [0, 3]
B_PRIME_POS = [2, 3]
DISCOUNT = 0.9

In [38]:
ACTIONS = [np.array([0, -1]),
           np.array([-1, 0]),
           np.array([0, 1]),
           np.array([1, 0])]

PROB = 0.25
theta = 1e-5

In [39]:
def is_terminal(state):
    x, y = state
    return (x==0 and y==0) or (x ==WORLD_SIZE -1 and y == WORLD_SIZE-1)

In [40]:
def step(state, action):
    if state == A_POS:
        return A_PRIME_POS, 10
    if state == B_POS:
        return B_PRIME_POS, 5

    next_state = (np.array(state) + action).tolist()
    x, y = next_state
    if x < 0 or x >= WORLD_SIZE or y < 0 or y >= WORLD_SIZE:
        reward = -1.0
        next_state = state
    else:
        reward = 0
    return next_state, reward

In [41]:
def compute_state_value():
    state_values = np.zeros((WORLD_SIZE, WORLD_SIZE))
    iteration = 0
    while True:
        delta = 0
        for i in range(WORLD_SIZE):
            for j in range(WORLD_SIZE):
                if is_terminal([i, j]):
                    continue
                value = 0
                for action in ACTIONS:
                    (next_i, next_j), reward = step([i, j], action)
                    value += PROB * (reward + state_values[next_i, next_j])
                delta = np.maximum(delta, np.abs(value - state_values[i, j]))
                state_values[i, j] = value
                
        iteration +=1
        if delta < theta:
            break
    
    return state_values, iteration

In [42]:
A = -1 * np.eye(WORLD_SIZE * WORLD_SIZE)
b = np.zeros(WORLD_SIZE * WORLD_SIZE)

for i in range (WORLD_SIZE):
    for j in range (WORLD_SIZE):
        s = [i, j]  # current state
        index_s = np.ravel_multi_index(s, (WORLD_SIZE, WORLD_SIZE))
        for a in ACTIONS:
            s_, r = step(s, a)
            index_s_ = np.ravel_multi_index(s_, (WORLD_SIZE, WORLD_SIZE))

            A[index_s, index_s_] += PROB * DISCOUNT
            b[index_s] -= PROB * r

    x = np.linalg.solve(A, b)

In [43]:
print(x)

[ 3.30899634  8.78929186  4.42761918  5.32236759  1.49217876  1.52158807
  2.99231786  2.25013995  1.9075717   0.54740271  0.05082249  0.73817059
  0.67311326  0.35818621 -0.40314114 -0.9735923  -0.43549543 -0.35488227
 -0.58560509 -1.18307508 -1.85770055 -1.34523126 -1.22926726 -1.42291815
 -1.97517905]
